## Implementing RAG with ChromaDB

### Documentation
https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/chroma/Using_Chroma_for_embeddings_search.ipynb

https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/chroma/hyde-with-chroma-and-openai.ipynb

In [4]:
#### When installing ChromaDB library it installs numpy-1.26.4 (deletes numpy 2.0.1). After installing ChromaDB library restart the kernel
#https://pypi.org/project/numpy/1.26.4/
#https://pypi.org/project/chromadb/0.5.5/

### Loading the data

In [5]:
# Importing libraries
import pandas as pd
import os
import ast

# Function to set the wd as the root of the repository
def find_repo_root(repo_name):
    current_dir = os.getcwd()
    while current_dir != '/':
        if os.path.basename(current_dir) == repo_name:
            return current_dir
        current_dir = os.path.dirname(current_dir)
    raise FileNotFoundError(f"Repository root '{repo_name}' not found.")

# Setting the working directory
repo_name = 'orare-model'
repo_root = find_repo_root(repo_name)
os.chdir(repo_root)

In [6]:
# Reading the bible interpreted with embeddings
bible_data = pd.read_csv('bible/data/bible_by_theme_int_embedding.txt', sep='|', encoding='utf-8')

# Tranforming vector string into list
bible_data['interpretacion_vector'] = bible_data['interpretacion_vector'].apply(ast.literal_eval)

# Showing the bible_data object
bible_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     861 non-null    object
 1   pasaje                 861 non-null    object
 2   texto                  861 non-null    object
 3   interpretación         861 non-null    object
 4   temas                  861 non-null    object
 5   área_vida              861 non-null    object
 6   texto_vector           861 non-null    object
 7   interpretacion_vector  861 non-null    object
dtypes: object(8)
memory usage: 53.9+ KB


### Setting ChromaDB Database and OpenAI client

In [7]:
# Importing libraries to set the connection to ChatGPT API and create a ChromaDB Client
import openai
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [8]:
# Instantiate the Chroma client
chroma_client = chromadb.EphemeralClient() # Equivalent to chromadb.Client(), ephemeral.

# Loading OpenAI api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Setting the embedding function for the query input
EMBEDDING_MODEL = "text-embedding-3-small"
embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name=EMBEDDING_MODEL)

In [9]:
# Creating or retrieving the collection in ChromaDB
try:
    # Try to create the collection (this will raise an error if it already exists)
    bible_interpretation_collection = chroma_client.create_collection(name='bible_interpretation', embedding_function=embedding_function)
except Exception as e:
    # If an error occurs (assuming it's because the collection already exists), retrieve it
    if 'Collection bible_interpretation already exists' in str(e):
        bible_interpretation_collection = chroma_client.get_collection(name='bible_interpretation')
    else:
        raise e


In [10]:
# Adding the embeddings to the collection
bible_interpretation_collection.add(
    ids=bible_data.id.tolist(),
    embeddings=bible_data.interpretacion_vector.tolist()
)

### Creating the Bible Verse Recommender

#### Helper Functions

In [11]:
# Function to search in the collection and return the results
def query_collection(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':results['distances'][0],
                'pasaje': dataframe[dataframe.id.isin(results['ids'][0])]['pasaje'],
                'texto': dataframe[dataframe.id.isin(results['ids'][0])]['texto'],
                'interpretación': dataframe[dataframe.id.isin(results['ids'][0])]['interpretación'],
                'temas': dataframe[dataframe.id.isin(results['ids'][0])]['temas'],
                'área_vida': dataframe[dataframe.id.isin(results['ids'][0])]['área_vida'],
                })
    
    return df

In [12]:
# Setting the connection with OpenAI API
openai_client = openai.OpenAI()

# Function to process a prompt in ChatGPT
def get_completion_4o(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [13]:
# Functions to process and parse the texn

# Function to flatten the text in a single line
def flatten_text(text):
    flattened_text = text.replace('\n', ' ')
    flattened_text = ' '.join(flattened_text.split())
    return flattened_text

# Function to insert line breaks
def insert_line_breaks(text, limit):
    lines = []
    while len(text) > limit:
        breakpoint = text.rfind(' ', 0, limit)
        if breakpoint == -1:  # No space found, force break
            breakpoint = limit
        lines.append(text[:breakpoint])
        text = text[breakpoint:].lstrip()  # Remove leading spaces from the remaining text
    lines.append(text)
    return '\n'.join(lines)

# Function to separate the paragraphs before insert line breaks
def format_text_with_line_breaks(text, limit):
    paragraphs = text.split('\n\n')
    formatted_paragraphs = [insert_line_breaks(paragraph, limit) for paragraph in paragraphs]
    return '\n\n'.join(formatted_paragraphs)

#### Verse Recommender Function

In [14]:
# Function to process the output of the Vector search and generate a prompt for ChatGPT to generate the final output for the user
def verse_recommender(prayer):
    # Flattening the prayer
    prayer = flatten_text(prayer)
    # Vector database search
    title_query_result = query_collection(
        collection=bible_interpretation_collection,
        query=prayer,
        max_results=3,
        dataframe=bible_data)
    
    # Prompt with Bible's verses resulted from the vector search
    # In this Prompt we take as input the "prayer" and the "Bible's verses recommened"
    # The ouptut is the interpretation of the "Bible's verse recommened" according to the prayer and some words of encouragement
    prompt = f"""
                # Rol #
                Tu eres el GPT: Catholic Bible Guide by Fr. Abraham Mutholath.
                Tu rol será el de un sacerdote de una iglesia Católica. Se te dará una oración de una persona y 3 pasajes de la Biblia relacionados a la oración. 
                Tu tarea será elegir el pasaje de la Biblia que más relación tenga con la oración de la persona. Luego deberás interpretar el pasaje de la Biblia que elegiste acorde a la oración de la persona y dar unas palabras de aliento.
                # Fin Rol #

                # Tareas #
                - Leer la oración de la persona y los 3 pasajes de la Biblia.
                - Elegir el pasaje de la Biblia que más relación tenga con la oración de la persona.
                - Interpretar el pasaje de la Biblia acorde a la oración de la persona y dar unas palabras de aliento.
                # fin Tareas #

                # Estructura input #
                - Oración de la persona
                - Pasaje de la Biblia 1
                - Pasaje de la Biblia 2
                - Pasaje de la Biblia 3
                # Fin estructura input #

                # Input #
                - Oración: {prayer}
                - Pasaje de la Biblia 1: {title_query_result.iloc[0,2]} :  {title_query_result.iloc[0,3]}
                - Pasaje de la Biblia 2: {title_query_result.iloc[1,2]} :  {title_query_result.iloc[1,3]}
                - Pasaje de la Biblia 3: {title_query_result.iloc[2,2]} :  {title_query_result.iloc[2,3]}
                # fin Input #

                # Estructura Output #
                versiculo elegido
                Párrafo con la interpretación del pasaje de la Biblia y palabras de aliento
                # Fin Estructura Output #

                # Ejemplo Output #
                Salmo 22:24 Porque no menospreció ni abominó la aflicción del pobre, Ni de él escondió su rostro; Sino que cuando clamó á él, oyóle.

                Este Salmo nos recuerda la infinita misericordia y compasión de Dios hacia los pobres y afligidos. Este versículo nos asegura que Dios no ignora el sufrimiento de los necesitados, ni les da la espalda. Al contrario, Él escucha sus clamores y está presente en sus momentos de angustia.
                Tu oración por los pobres del mundo es un acto de amor y solidaridad que refleja el corazón de Dios. Al interceder por ellos, te unes a la misión de Cristo de traer consuelo y esperanza a los más vulnerables. Recuerda que Dios escucha nuestras oraciones y actúa a través de nosotros para llevar su amor y provisión a aquellos que más lo necesitan.
                Te animo a seguir orando y, si es posible, a tomar acciones concretas para ayudar a los pobres en tu comunidad. Cada pequeño gesto de generosidad y compasión puede ser una manifestación del amor de Dios en sus vidas. Confía en que Dios, en su infinita bondad, no abandonará a los necesitados y usará nuestras oraciones y acciones para bendecirlos.
                Que el Señor te bendiga y te fortalezca en tu deseo de servir a los demás. Amén.
                # Fin Ejemplo Ouput #
              """
    # Function to process the prompt
    response_4o = get_completion_4o(prompt)
    # Formatting the answer of ChatGPT
    response_4o = response_4o.lstrip()
    response_4o_break = format_text_with_line_breaks(response_4o, 150)
    # Formatting the prayer
    prayer_break = format_text_with_line_breaks(prayer, 150)
    # Formatting the Bible verses and its interpretation from the Vector Search
    verse1 = format_text_with_line_breaks(title_query_result.iloc[0,3],130)
    verse2 = format_text_with_line_breaks(title_query_result.iloc[1,3],130)
    verse3 = format_text_with_line_breaks(title_query_result.iloc[2,3],130)
    inter1 = format_text_with_line_breaks(title_query_result.iloc[0,4],150)
    inter2 = format_text_with_line_breaks(title_query_result.iloc[1,4],150)
    inter3 = format_text_with_line_breaks(title_query_result.iloc[2,4],150)
    # Printing all the verses recommened in order to check the process (For developer's use only)
    print('Pasajes recomendados')
    print('---------------------------')
    print('- '+title_query_result.iloc[0,2] + ': ' + verse1)
    print('- Interpretación: ' + inter1)
    print('---------------------------')
    print('- '+title_query_result.iloc[1,2] + ': ' + verse2)
    print('- Interpretación: ' + inter2)
    print('---------------------------')
    print('- '+title_query_result.iloc[2,2] + ': ' + verse3)
    print('- Interpretación: ' + inter3)
    # Printing the final output of the chat
    print('---------------------------')
    print('Output Orare')
    print('---------------------------')
    print(f'Oración: {prayer_break}')
    print('---------------------------')
    print(response_4o_break)

    return

### Testing 1st MVP Orare - ChromaDB

In [ ]:
prayer = "Dios, te pido y oro por los pobres del mundo que no tienen que comer. Por favor se misericordioso con ellos. Pido por ellos. Amen"
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Deuteronomio 15:11: Porque no faltarán menesterosos de en medio de la tierra; por eso yo te mando, diciendo: Abrirás tu mano á tu hermano, á tu
pobre, y á tu menesteroso en tu tierra.
- Interpretación: Este versículo nos recuerda que siempre habrá personas necesitadas entre nosotros. Por lo tanto, Dios nos manda ser generosos y ayudar a nuestros
hermanos, especialmente a los pobres y necesitados.
---------------------------
- Malaquías 3:10: Traed todos los diezmos al alfolí, y haya alimento en mi casa; y probadme ahora en esto, dice Jehová de los ejércitos, si no os
abriré las ventanas de los cielos, y vaciaré sobre vosotros bendición hasta que sobreabunde.
- Interpretación: Dios está pidiendo a su pueblo que traigan los diezmos (una décima parte de sus ingresos) al templo para que haya suficiente provisión. Él promete
que si obedecen, abrirá las ventanas del cielo y derramará bendiciones abundantes sobre ellos.
----------------------

In [ ]:
prayer = '''Querido Dios, 
Hoy me encuentro angustiado y ansioso por recibir un email que me puede cambiar la vida. Estoy a la espera de una noticia muy importante para mí, 
pero mi falta de paciencia y de quererlo todo ahora ya, hace que me angustie y estrese. 
Por otro lado, hoy he vuelto ir a la fisio por un dolor que tuve en la pierna y ya estoy mejor, 
el estres hace que me tense mucho y mis musculos acaben contracturandose demasio.
Te pido que me des paciencia en este momento, que tenga confianza en que todo va a salir bien, 
y lo que tú quieres para mí se va cumplir tal y como tú lo planeas.
Amén.'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Salmos 27:13, 14: Hubiera yo desmayado, si no creyese que tengo de ver la bondad de Jehová En la tierra de los vivientes. Aguarda á Jehová;
Esfuérzate, y aliéntese tu corazón: Sí, espera á Jehová.
- Interpretación: El salmista expresa que habría perdido la esperanza si no creyera en la bondad de Dios en esta vida. Anima a tener paciencia y a confiar en el Señor,
fortaleciendo el corazón mientras se espera en Él.
---------------------------
- Salmos 27:14: Aguarda á Jehová; Esfuérzate, y aliéntese tu corazón: Sí, espera á Jehová.
- Interpretación: Este versículo nos anima a tener paciencia y confianza en Dios. Nos exhorta a ser fuertes y a mantener el ánimo mientras esperamos en el Señor.
---------------------------
- Salmos 40:13: Quieras, oh Jehová, librarme; Jehová, apresúrate á socorrerme.
- Interpretación: El salmista está pidiendo a Dios que lo libere de sus problemas y que lo ayude rápidamente. Es una súplica urgente por ayuda di

In [ ]:
prayer = '''Querido Dios,
Hoy ha sido un gran día. He vuelto de un viaje de Madrid con mi novia, y ha ido genial me lo he pasado muy bien, 
aunque he estado cansado durante el día. Y he tenido la noticia, de que ya empezaré a trabajar en una nueva empresa en septiembre,
con la cual estoy muy motivado y con muchas ganas de empezar. Esto significa, que debo de poner la renuncia en la actual empresa,
dame la serenidad y calma para hacerlo de la mejor manera dando gracias por la oportunidad que me han dado, 
pero se me presentó una oportunidad irrechazable. Espero que no se lo tomen mal, intentaré decirles con las mejores palabras y respeto. 
También, hoy ha sido el cumple de mi padre, hemos pasado una bonita cena en familia, con los míos.
Muchas gracias por otro día más de vida'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Job 11:18, 19: Y confiarás, que habrá esperanza; Y cavarás, y dormirás seguro: Y te acostarás, y no habrá quien te espante: Y muchos te rogarán.
- Interpretación: Este pasaje habla de la confianza en Dios y la esperanza que trae seguridad y paz. Se menciona que, al confiar en Dios, uno puede descansar sin temor
y que otros buscarán tu favor.
---------------------------
- Juan 14:28: Habéis oído cómo yo os he dicho: Voy, y vengo á vosotros. Si me amaseis, ciertamente os gozaríais, porque he dicho que voy al
Padre: porque el Padre mayor es que yo.
- Interpretación: Jesús está hablando a sus discípulos sobre su partida y regreso. Les dice que deberían alegrarse porque va al Padre, quien es mayor que Él. Esto
subraya la relación especial entre Jesús y Dios Padre, y la necesidad de confiar en el plan divino.
---------------------------
- Salmos 112:7, 8: De mala fama no tendrá temor: Su corazón está apercibido, confiado en Jehová. Asentado 

In [ ]:
prayer = '''Le pido a Dios me de claridad en cuanto a mi objetivo a largo plazo profesional/laboral'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Salmos 119:18: Abre mis ojos, y miraré Las maravillas de tu ley.
- Interpretación: El salmista está pidiendo a Dios que le dé entendimiento y claridad para poder apreciar y comprender las maravillas y enseñanzas contenidas en la ley
divina.
---------------------------
- Salmos 143:10, 11: Enséñame á hacer tu voluntad, porque tú eres mi Dios: Tu buen espíritu me guíe á tierra de rectitud. Por tu nombre, oh Jehová me
vivificarás: Por tu justicia, sacarás mi alma de angustia.
- Interpretación: El salmista pide a Dios que le enseñe a hacer Su voluntad y que Su Espíritu lo guíe por el camino correcto. También pide ser revivido y liberado de
la angustia por la justicia de Dios.
---------------------------
- Salmos 5:8: Guíame, Jehová, en tu justicia á causa de mis enemigos; Endereza delante de mí tu camino.
- Interpretación: El salmista pide a Dios que lo guíe en justicia y que le muestre el camino correcto, especialmente debido a la presenc

In [ ]:
prayer = '''No se que podrá pasar mañana, pero confío en ti. Nunca he estado en tanta sintonía con mi presente, 
con esta adrenalina de ver la primera cima, de empezar a estirar los brazos para cruzar la meta y 
de esta fe ciega de que las cosas se van a ir poniendo en su lugar. Hoy la vida me vuelve a enseñar, 
que nada está en mi control, que todo depende de ti. Y por eso acudo una vez más a tu consejo, 
guíame señor, dame esa luz para poder convencer si son esos tus planes, muéstrame el camino, 
dame la esperanza para luchar esta batalla una vez más, para inflar el pecho por creer que todo es posible,
pero que es posible junto a ti. Pero te pido también, con las manos temblando del temor, que si esta puerta no es para mi, 
la cierres por completo, aleja de mi lo que a la larga no sea para tu provecho, si este sendero no he de recorrer,
cámbiame el camino, quiebra la oportunidad, porque prefiero perder incluso mis sueños, pero jamás perderte a ti,
porque una vida de triunfos es vacío si no estás a mi lado, porque aún en la derrota, hay paz si es estás junto a mi,
porque hoy más que nunca, te digo de rodillas, que se haga en mi según tu palabra, que se haga en mi según tu voluntad 
y te ruego me des la sabiduría para entender, cualquiera que sea tu designio. Amén'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Job 11:18, 19: Y confiarás, que habrá esperanza; Y cavarás, y dormirás seguro: Y te acostarás, y no habrá quien te espante: Y muchos te rogarán.
- Interpretación: Este pasaje habla de la confianza en Dios y la esperanza que trae seguridad y paz. Se menciona que, al confiar en Dios, uno puede descansar sin temor
y que otros buscarán tu favor.
---------------------------
- Salmos 5:3: Oh Jehová, de mañana oirás mi voz; De mañana me presentaré á ti, y esperaré.
- Interpretación: El salmista está expresando su devoción y confianza en Dios, indicando que cada mañana se presentará ante Él en oración y esperará con esperanza y fe.
---------------------------
- Salmos 61:2: Desde el cabo de la tierra clamaré á ti, cuando mi corazón desmayare: A la peña más alta que yo me conduzcas.
- Interpretación: El salmista está expresando su desesperación y su necesidad de ayuda divina. "Desde el cabo de la tierra" sugiere que se siente muy lejos o en una

In [ ]:
prayer = '''Ilumíname señor, dame paz en estos momentos de incertidumbre, sujétame de tu mano, permíteme caminar junto a ti.
¿Por qué permites este dolor dentro de nosotros? ¿Por qué este sufrimiento tan profundo? No lograré entender tus motivos, 
pero entiendo espiritualmente lo que estás haciendo conmigo.
Purificas mi alma, me permitiste soltar el control, afianzarme a ti, volver a mirarte cara a cara, 
entender que todo depende de ti, reconocer la vanidad de mis actos, sentir miedo a cada instante y que mi única fuerza seas tú.
No entiendo tus caminos pero decido caminar dentro de ellos, seguir tu luz, guiar a mi familia hacia tus designios, 
respirar en esa absoluta incertidumbre y orar con todas mis fuerzas para que se haga según tu voluntad.
Cuida a mi familia señor, bendice a mi esposa, cúbrela con tu manto sagrado, fortalece a mi hijo, 
aliéntalo con tu espíritu, dale el soporte que hoy nosotros no podemos darle, 
que su angel guardián le cuente que estamos todos pendientes de él, 
que no contamos los minutos para tenerlo en nuestros brazos, pero que sea paciente,
que tú tienes varios corazones que limpiar en estos momentos, que su venida está generando santidad en nuestro alrededor.'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- 2 Samuel 22:5-7: Cuando me cercaron ondas de muerte, Y arroyos de iniquidad me asombraron, Me rodearon los dolores del infierno, Y me tomaron
descuidado lazos de muerte. Tuve angustia, invoqué á Jehová, Y clamé á mi Dios: Y él oyó mi voz desde su templo; Llegó mi clamor
á sus oídos.
- Interpretación: El fragmento describe una situación de gran angustia y peligro, donde el autor se siente rodeado por la muerte y el mal. En su desesperación, clama a
Dios, quien escucha su súplica y responde. Este pasaje resalta la importancia de la fe y la confianza en Dios en momentos de crisis.
---------------------------
- Job 11:18, 19: Y confiarás, que habrá esperanza; Y cavarás, y dormirás seguro: Y te acostarás, y no habrá quien te espante: Y muchos te rogarán.
- Interpretación: Este pasaje habla de la confianza en Dios y la esperanza que trae seguridad y paz. Se menciona que, al confiar en Dios, uno puede descansar sin temor
y que otros buscarán 

In [ ]:
prayer = '''Dios. Quiero que en este proceso de búsqueda seas mi fuerza y me acompañes en todo momento. 
Yo se que tienes para mi un plan. Yo se que tu siempre das en la medida correcta y que poco a poco me has mostrado el camino para mi crecimiento.
Hace un tiempo me faltaba confianza en mi mismo. Se que con la experiencia laboral anterior me diste la oportunidad de abrir los ojos y saber quien soy.
Te doy las gracias porque se que cada paso que me muestras es un paso hacia desarrollar mi máximo potencial.
'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Jeremías 29:11-13: Porque yo sé los pensamientos que tengo acerca de vosotros, dice Jehová, pensamientos de paz, y no de mal, para daros el fin que
esperáis. Entonces me invocaréis, é iréis y oraréis á mí, y yo os oiré: Y me buscaréis y hallaréis, porque me buscaréis de todo
vuestro corazón.
- Interpretación: Dios asegura a su pueblo que tiene planes de bienestar y no de mal para ellos, prometiendo un futuro esperanzador. Les invita a buscarlo sinceramente
a través de la oración, asegurándoles que los escuchará y se dejará encontrar.
---------------------------
- Job 11:18, 19: Y confiarás, que habrá esperanza; Y cavarás, y dormirás seguro: Y te acostarás, y no habrá quien te espante: Y muchos te rogarán.
- Interpretación: Este pasaje habla de la confianza en Dios y la esperanza que trae seguridad y paz. Se menciona que, al confiar en Dios, uno puede descansar sin temor
y que otros buscarán tu favor.
---------------------------
- Prove

In [ ]:
prayer = '''Dios en esta oportunidad quiero pedirte que me des luz y fuerzas para poder continuar con la búsqueda de trabajo que estoy haciendo. 
Se que es un poco dificil conseguir trabajo como extranjero, pero no tengo dudas de que Tú me ayudarás y pondrás la oportunidad ante mis ojos. 
En el nombre del Padre, del Hijo, del Espíritu Santo, Amén.'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Lucas 11:9, 10: Y yo os digo: Pedid, y se os dará; buscad, y hallaréis; llamad, y os será abierto. Porque todo aquel que pide, recibe; y el que
busca, halla; y al que llama, se abre.
- Interpretación: Jesús está enseñando sobre la importancia de la perseverancia en la oración. Nos anima a pedir, buscar y llamar, asegurándonos que Dios responderá a
nuestras peticiones, búsquedas y llamados.
---------------------------
- Mateo 7:7: Pedid, y se os dará; buscad, y hallaréis; llamad, y se os abrirá.
- Interpretación: Este versículo nos enseña sobre la importancia de la perseverancia en la oración y la fe. Jesús nos anima a pedir, buscar y llamar, asegurándonos que
Dios responderá a nuestras necesidades y deseos si lo hacemos con fe y constancia.
---------------------------
- Romanos 8:26, 27: Y asimismo también el Espíritu ayuda nuestra flaqueza: porque qué hemos de pedir como conviene, no lo sabemos; sino que el mismo
Espíritu pide por nos

In [ ]:
oracion = '''Señor bendito esta mañana t agradezco la salud y la vida
Que tu gracia y misericordia sean hoy conmigo 
Padre t ruego fortaleza  en todo momento 
Que tu luz siempre  m acompañe
Perdóname en mis momentos d cólera  
Ayúdame a tener paciencia  y amor para mi y mis semejantes '''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Lucas 11:9, 10: Y yo os digo: Pedid, y se os dará; buscad, y hallaréis; llamad, y os será abierto. Porque todo aquel que pide, recibe; y el que
busca, halla; y al que llama, se abre.
- Interpretación: Jesús está enseñando sobre la importancia de la perseverancia en la oración. Nos anima a pedir, buscar y llamar, asegurándonos que Dios responderá a
nuestras peticiones, búsquedas y llamados.
---------------------------
- Mateo 7:7: Pedid, y se os dará; buscad, y hallaréis; llamad, y se os abrirá.
- Interpretación: Este versículo nos enseña sobre la importancia de la perseverancia en la oración y la fe. Jesús nos anima a pedir, buscar y llamar, asegurándonos que
Dios responderá a nuestras necesidades y deseos si lo hacemos con fe y constancia.
---------------------------
- Romanos 8:26, 27: Y asimismo también el Espíritu ayuda nuestra flaqueza: porque qué hemos de pedir como conviene, no lo sabemos; sino que el mismo
Espíritu pide por nos

In [ ]:
prayer = ''' Dios mío gracias por un nuevo día, donde no me falta salud ni a mí ni a los míos. 
Te pido me des las fuerzas para seguir de pie y no rendirme en este nuevo camino que he tomado fuera de mi país, 
migrar no es fácil, lo sé, y extraño tanto a mi tierra y los míos que a veces se van las fuerzas de seguir luchando. 
Estoy aquí a pesar de recibir miles des “NO” y de que me hayan rechazado mi visado para estar aquí en España… 
será que sigo luchando o tú ya tienes marcado un destino para mí en otro lugar ?'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Isaías 40:29-31: El da esfuerzo al cansado, y multiplica las fuerzas al que no tiene ningunas. Los mancebos se fatigan y se cansan, los mozos
flaquean y caen: Mas los que esperan á Jehová tendrán nuevas fuerzas; levantarán las alas como águilas, correrán, y no se
cansarán, caminarán, y no se fatigarán.
- Interpretación: Este pasaje habla de cómo Dios da fuerza y energía a aquellos que están cansados y débiles. Incluso los jóvenes pueden cansarse y desfallecer, pero
aquellos que confían en el Señor recibirán nuevas fuerzas y podrán seguir adelante sin agotarse.
---------------------------
- Josúe 1:9: Mira que te mando que te esfuerces y seas valiente: no temas ni desmayes, porque Jehová tu Dios será contigo en donde quiera que
fueres.
- Interpretación: Este versículo es una exhortación a ser fuerte y valiente, recordando que Dios siempre estará con nosotros, sin importar dónde estemos. Es un llamado
a confiar en la presencia y apoyo co

In [ ]:
prayer = ''' Dios, quiero pedirte por Salvador, el hijo de Sergio. 
Tengo un gran afecto por Sergio que es un amigo con el cual ya tengo una amistad de más de 10 años.
Se que está pasando por momentos difíciles. Todos esperabamos con mucha alegría y emoción la llegada de su primogénito. 
Sin embargo, tuvieron problemas con el parto y Salvador nació prematuro.
Salvador está en cuidados intensivos y algunos días avanzando, otros retrocediento. 
Te pido con todo mi corazón que bendigas y protejas al pequeño Salvador.
Sé que nada está en nuestras manos y que todo dependerá de Tu Voluntad.
'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Salmos 31:2: Inclina á mí tu oído, líbrame presto; Séme por roca de fortaleza, por casa fuerte para salvarme.
- Interpretación: El salmista está pidiendo a Dios que lo escuche y lo salve rápidamente, comparando a Dios con una roca fuerte y una casa segura que lo protege.
---------------------------
- Salmos 40:13: Quieras, oh Jehová, librarme; Jehová, apresúrate á socorrerme.
- Interpretación: El salmista está pidiendo a Dios que lo libere de sus problemas y que lo ayude rápidamente. Es una súplica urgente por ayuda divina.
---------------------------
- Salmos 40:17: Aunque afligido yo y necesitado, Jehová pensará de mí: Mi ayuda y mi libertador eres tú; Dios mío, no te tardes.
- Interpretación: El salmista expresa su confianza en Dios a pesar de estar en una situación de aflicción y necesidad. Reconoce a Dios como su ayuda y libertador, y
pide que no tarde en socorrerlo.
---------------------------
Output Orare
-----------------------

In [ ]:
prayer = '''Querido Dios,
Hoy me acerco a Ti con un corazón humilde y lleno de esperanza. Te pido por Salvador, el hijo de mi querido amigo Sergio. 
Tú conoces la situación delicada en la que se encuentra, nacido prematuramente y luchando cada día en cuidados intensivos.
Señor, te ruego que pongas Tu mano sanadora sobre él. Dale la fortaleza necesaria para superar cada obstáculo y que su salud mejore día a día. 
Que cada avance sea un paso firme hacia su recuperación completa.
Brinda a los médicos y enfermeras que lo cuidan la sabiduría y la habilidad para atenderlo de la mejor manera posible. 
Asimismo, te pido que fortalezcas a Sergio y a su familia en estos momentos difíciles, dándoles paz, esperanza y la certeza de Tu amor y presencia constante.
Confiamos en Tu infinita bondad y misericordia, sabiendo que en Ti todo es posible. Te agradecemos por escuchar nuestras súplicas y por estar siempre a nuestro lado.
Amén.'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Salmos 143:8: Hazme oir por la mañana tu misericordia, Porque en ti he confiado: Hazme saber el camino por donde ande, Porque á ti he alzado mi
alma
- Interpretación: El salmista está pidiendo a Dios que le muestre su misericordia y guía desde temprano en la mañana, confiando plenamente en Él. Está buscando
dirección y apoyo espiritual, elevando su alma a Dios en señal de dependencia y fe.
---------------------------
- Salmos 40:13: Quieras, oh Jehová, librarme; Jehová, apresúrate á socorrerme.
- Interpretación: El salmista está pidiendo a Dios que lo libere de sus problemas y que lo ayude rápidamente. Es una súplica urgente por ayuda divina.
---------------------------
- Salmos 40:17: Aunque afligido yo y necesitado, Jehová pensará de mí: Mi ayuda y mi libertador eres tú; Dios mío, no te tardes.
- Interpretación: El salmista expresa su confianza en Dios a pesar de estar en una situación de aflicción y necesidad. Reconoce a Dios como s

In [ ]:
prayer = '''
Dios en esta oración quiero pedirte por mi mamá. Ella falleció hace 2 años de cáncer.
Durante los últimos meses de la enfermedad ella estuvo sufriendo mucho. Yo te pedí en esa ocasión que si no había más remedio lo que menos quería era que sufriera.
El cáncer es una enfermedad muy dolorosa y con mucho sufrimiento. Lo que menos quería es que ella sufriera más porque es más dolorosó ver a un familiar cercano pasando por dicha situación. 
Te doy las gracias por los buenos momentos que hemos pasado en familia. Por ese último viaje que hiciemos a USA donde vive mi hermana. 
Te doy las gracias por esas fotos que pudimos tomarnos disfrutando sus últimos momentos de lucidez y salud.
Te doy las gracias por haberme dado a esa madre que siempre estuvo atenta de mí y cuidandome cuando era un niño y me enfermaba con frecuencia.
Te doy las gracias por que no pudo tocarme una madre más amorosa y preocupada que sus hijos salgan adelante.
Te pido que la tengas en tu Gloria cada instante y que este tranquila. Ella ha criado correctamente a sus hijos y nos va a ver desde el Cielo haciendo el bien y saliendo adelante.
Te pido que por favor su espíritu siempre este a mi lado acompañandome en cada paso y cada reto que estoy pasando.
Quiero que en cada logro ella este presente disfrutando lo que sembró como madre.
'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- 2 Colosenses 3:5: Y el Señor enderece vuestros corazones en el amor de Dios, y en la paciencia de Cristo.
- Interpretación: Este versículo es una oración para que el Señor guíe y fortalezca los corazones de los creyentes en el amor de Dios y en la paciencia que Cristo
mostró. Se trata de una petición para que los fieles puedan vivir de acuerdo con estos valores.
---------------------------
- 2 Corintios 2:14: Mas á Dios gracias, el cual hace que siempre triunfemos en Cristo Jesús,
- Interpretación: Este versículo agradece a Dios por darnos la victoria a través de Cristo Jesús. La idea es que, con la ayuda de Cristo, siempre podemos superar las
dificultades y triunfar en nuestras vidas.
---------------------------
- Filipenses 4:6: Por nada estéis afanosos; sino sean notorias vuestras peticiones delante de Dios en toda oración y ruego, con hacimiento de
gracias.
- Interpretación: Este versículo nos aconseja no preocuparnos excesivamente

In [ ]:
prayer = '''
Dios, hoy me acerco a ti para pedirte por Salvador, el hijo de mi amigo Sergio. 
Tu sabes la situación delicada que está atravesando. Quiero pedirte por su salud y porque cada paso que de sea un paso firme hacia su recuperación total.
Te pido porque bendigas a Sergio y su familia y le des la fortaleza para superar esta situación.
Dale la sabiduría a los médicos y enfermeras que cuidan de él.
Sé que todo está en Tus manos y que en Ti todo es posible.
Te agradecemos por escuchar nuestras súplicas y por siempre acompañarnos.
Amén.
'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Salmos 143:10, 11: Enséñame á hacer tu voluntad, porque tú eres mi Dios: Tu buen espíritu me guíe á tierra de rectitud. Por tu nombre, oh Jehová me
vivificarás: Por tu justicia, sacarás mi alma de angustia.
- Interpretación: El salmista pide a Dios que le enseñe a hacer Su voluntad y que Su Espíritu lo guíe por el camino correcto. También pide ser revivido y liberado de
la angustia por la justicia de Dios.
---------------------------
- Salmos 25:4, 5: Muéstrame, oh Jehová, tus caminos; Enséñame tus sendas. Encamíname en tu verdad, y enséñame; Porque tú eres el Dios de mi salud:
En ti he esperado todo el día.
- Interpretación: El salmista está pidiendo a Dios que le guíe y le enseñe el camino correcto, buscando la verdad y la salvación que solo Dios puede ofrecer. Es una
expresión de confianza y dependencia en Dios.
---------------------------
- Salmos 40:13: Quieras, oh Jehová, librarme; Jehová, apresúrate á socorrerme.
- Interpretació

In [ ]:
prayer = '''
Dios, estos días he estado avanzando en varios procesos de selección. Varias de estas ofertas laborales son muy atractivas.
Asimismo también recibí noticias de otros procesos de selección en el cuál no fui elegido.
En los días siguientes tendré varias entrevistas importantes. He estado trabajando y reflexionando en las mejoras que debo aplicar en las siguientes entrevistas.
Confío en Tu Sabiduría infinita y en que cada paso, cada fallo, cada aprendizaje solo es un paso más para eso que tienes preparado para mi.
Quiero que me acompañes en cada momento de este proceso y que me des la calma, paz y temple para afrontar el siguiente paso.
Confío en Ti y en que sabes que es lo mejor para mí.
Gracias por todo.
Amén.
'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Jeremías 29:11-13: Porque yo sé los pensamientos que tengo acerca de vosotros, dice Jehová, pensamientos de paz, y no de mal, para daros el fin que
esperáis. Entonces me invocaréis, é iréis y oraréis á mí, y yo os oiré: Y me buscaréis y hallaréis, porque me buscaréis de todo
vuestro corazón.
- Interpretación: Dios asegura a su pueblo que tiene planes de bienestar y no de mal para ellos, prometiendo un futuro esperanzador. Les invita a buscarlo sinceramente
a través de la oración, asegurándoles que los escuchará y se dejará encontrar.
---------------------------
- Job 11:18, 19: Y confiarás, que habrá esperanza; Y cavarás, y dormirás seguro: Y te acostarás, y no habrá quien te espante: Y muchos te rogarán.
- Interpretación: Este pasaje habla de la confianza en Dios y la esperanza que trae seguridad y paz. Se menciona que, al confiar en Dios, uno puede descansar sin temor
y que otros buscarán tu favor.
---------------------------
- Prove

In [15]:
prayer = '''
Dios Todopoderoso. Hoy es un día importante para mi y el futuro de mi carrera profesional.
Voy a tener dos entrevistas clave que son parte de mi búsqueda laboral para asentarme en este país.
Quisiera que me des el temple, la calma y paz para responder cada pregunta. 
Me he estado preparando mucho para afrontar estas dos entrevistas.
Se que tienes preparado algo para mi y aceptaré Tu Voluntad.
Quiero que me acompañes en este proceso, me bendigas y siempre estés conmigo.
Amén.
'''
verse_recommender(prayer)

Pasajes recomendados
---------------------------
- Isaías 45:11: Así dice Jehová, el Santo de Israel, y su Formador: Preguntadme de las cosas por venir; mandadme acerca de mis hijos, y acerca de
la obra de mis manos.
- Interpretación: El Señor, el Santo de Israel, está invitando a su pueblo a preguntarle sobre el futuro y a confiar en su plan y obra. Esto refleja la relación de
confianza y dependencia que deben tener con Dios.
---------------------------
- Jeremías 29:11-13: Porque yo sé los pensamientos que tengo acerca de vosotros, dice Jehová, pensamientos de paz, y no de mal, para daros el fin que
esperáis. Entonces me invocaréis, é iréis y oraréis á mí, y yo os oiré: Y me buscaréis y hallaréis, porque me buscaréis de todo
vuestro corazón.
- Interpretación: Dios asegura a su pueblo que tiene planes de bienestar y no de mal para ellos, prometiendo un futuro esperanzador. Les invita a buscarlo sinceramente
a través de la oración, asegurándoles que los escuchará y se dejará encontrar.